In [ ]:
# 1) Gets the username and password of an account with full access(read/write) to the SharePoint site. This username and password is saved in Azure key vault.

sharepoint_usr = dbutils.secrets.get(scope = 'Key-Vault-Secret', key = 'SharePoint-Acc')
sharepoint_pwd = dbutils.secrets.get(scope = 'Key-Vault-Secret', key = 'SharePoint-PWD')

In [ ]:
# 2) Connects to the SharePoint site, read the csv file and get the csv file into a pandas dataframe

# Install sharepy library seperately to the cluster used
# json and pandas libraries are by default available in the cluster, so no need to install seperately
import sharepy
import json
import pandas as pd

# Authenticate
s = sharepy.connect('<enter_sharepoint_site_url>',username=sharepoint_usr, password=sharepoint_pwd)

file_url = '<enter_csv_filepath_in_sharepoint_site>'

filename = 'Products.csv'

r = s.getfile(file_url, filename=filename)
df = pd.read_csv(filename)
df = spark.createDataFrame(df.astype(str))

In [ ]:
# 3) Prints the contents of the dataframe

display(df)

PRODUCTLINE,PRODUCTCODE
Motorcycles,S10_1678
Classic Cars,S10_1949
Motorcycles,S10_2016
Motorcycles,S10_4698
Classic Cars,S10_4757
Classic Cars,S10_4962
Classic Cars,S12_1099
Classic Cars,S12_1108
Trucks and Buses,S12_1666
Motorcycles,S12_2823


In [ ]:
# 4) Dbutils widget created to get the folder path to save the parquet file in the data lake. A value can be passed as a parameter for this widget.

# path -  dimension/product
dbutils.widgets.text('path', '')
path = dbutils.widgets.get('path')
print(path)

In [ ]:
# 5) Writes the dataframe in parquet format to the data lake

# Access key used to connect to the data lake
# This value is saved in the Azure key vault
str_key = dbutils.secrets.get(scope = 'Key-Vault-Secret', key = 'Data-Lake')

spark.conf.set(
  'fs.azure.account.key.heshtestdwhstorage.dfs.core.windows.net',
  str_key
)

# Data lake folder path where we want to write the parquet file
target_folder_path = 'abfss://data-lake@heshtestdwhstorage.dfs.core.windows.net/' + path

df.coalesce(1).write.mode('overwrite').option('header','false').parquet(target_folder_path)